### Word Group In Youtube

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 1000  # native word end index 28,200,1000,5000,10000, 20000, 40000

# word ratio
if word_end == 28 or word_end == 200:
    unique_word_ratio = 70
    include_wordgroup_ratio = 70
elif word_end == 1000 or word_end == 5000:
    unique_word_ratio = 80
    include_wordgroup_ratio = 80
elif word_end == 10000:
    unique_word_ratio = 85
    include_wordgroup_ratio = 85
else:
    unique_word_ratio = 90
    include_wordgroup_ratio = 90

# word usage
if word_end == 28 or word_end == 200 or word_end == 1000 or word_end == 5000 or word_end == 10000:
    word_usage_min = 10
    word_usage_max = 100
elif word_end == 20000:
    word_usage_min = 7
    word_usage_max = 100
else:
    word_usage_min = 5
    word_usage_max = 100

# use for manuel set
#unique_word_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 ; 10000 = 85 ; 20000,40000 = 90
#include_wordgroup_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 : 10000 = 85 ; 20000,40000 = 90
#word_usage_min = 10  # 28,200,1000,5000,10000 = 10 ; 20000 = 7 ; 40000 = 5
#word_usage_max = 100


# sentence ratio and time shift
adjust_sent_word_ratio = 15  #28,200 = 15 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
#adjust_text_word_ratio = 5  #28,200 = 5 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
shift = 0.3  # sentence time shift

# func select
if word_end == 28:
    sentence_word_count_number = 2
else:
    sentence_word_count_number = 5   
word_group_adjust_select = False  # True, False; True for selecting word group according to sentence_word_count_number in video_id text analysis
                                  # False for selecting word group according to max word group length 

# shared word frequency
word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"word_end:{word_end}\nfile_ext:{file_ext}\nunique_word_ratio:{unique_word_ratio}\n\
include_wordgroup_ratio:{include_wordgroup_ratio}\nword_usage_min:{word_usage_min}\nword_usage_max:{word_usage_max}")

word_end:1000
file_ext:6
unique_word_ratio:80
include_wordgroup_ratio:80
word_usage_min:10
word_usage_max:100


In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment3/Result/3-Word Group In Youtube"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by=f"{target_opt_column}", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [6]:
def exract_word_group(text, word_list):
    '''
    exract_word_group(text, word_list): \n
    text is a string sentence, word_list occurs words 
    '''
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []
    # check words in word_list
    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass
    # add index plus 1 and compare with current and next index (Is it continuous or not?)
    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []
    # index convert to word
    text_list = []
    for k in max(all_index_list, key=len):  # any error convert k to i
        word = words[k]
        text_list.append(word)
        if len(text_list) >= sentence_word_count_number:
            text = " ".join(text_list)
        else:
            text = f"sentencte_word_count_less_than_{sentence_word_count_number}"
    
    return text 

In [7]:
def df_exract_word_group(df, source_text_column, opt_column, word_list, sent_len=False, sent_len_num=2):
    '''
    df_exract_word_group(df_adjust_text_ratio, sentence, video_id, word_list, sent_len=False, sent_len_num=2): \n
    df_adjust_text_ratio is a dataframe and it includes sentence and video_id columns. \n
    sentence is a string sentence. video_id is optional column value. word_list occurs words \n
    sent_len is sentence length condition. sent_len_num is sentence occurs how many word at least. 
    '''
    df.reset_index(drop=True, inplace=True)
    df_new = pd.DataFrame()
    for i in range(len(df)):
        source_text = df.loc[i,f"{source_text_column}"]
        opt_var = df.loc[i,f"{opt_column}"]
        words = re.findall(r"\w+", source_text, re.UNICODE)
        # check words in word_list
        index_list = []
        for j in range(len(words)):
            if words[j] in word_list:
                index_list.append(j)
            else:
                pass
        # add index plus 1 and compare with current and next index (Is it continuous or not?)
        all_index_list = []
        var_index_list = []
        for k in range(len(index_list)):
            try:
                var1 = index_list[k] + 1  
                var2 = index_list[k+1]
            except:
                var1 = index_list[k] + 1  
                var2 = index_list[-1]
            if var1 == var2:
                var3 = index_list[k]
                var_index_list.append(var3)
            else:
                var3 = index_list[k]
                var_index_list.append(var3)
                var4 = list(var_index_list)
                all_index_list.append(var4)
                var_index_list = []
        # index convert to word
        text_all_list = []
        for m in all_index_list:
            text_list = [] 
            for n in m:
                word = words[n]
                text_list.append(word)
                if sent_len:
                    if len(text_list) >= sent_len_num:
                        text = " ".join(text_list)
                        text_all_list.append(text)
                    else:
                        pass
                else:
                   text = " ".join(text_list)
                   text_all_list.append(text) 
        # dataframe output     
        for search_string in text_all_list:
            #df_var["index"] = i
            #df_var["search_string"] = search_string
            #df_var["sentence"] = source_text
            #df_var["video_id"] = opt_var
            df_list = []
            df_list.append([i,search_string,source_text,opt_var])
            df_var = pd.DataFrame(df_list,columns=["index","search_string",f"{source_text_column}",f"{opt_column}"])
            df_new = pd.concat([df_new, df_var], axis=0)

    return df_new 

In [8]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [9]:
def repetation_ratio(text):
    '''
    repetation_ratio(text) 
    text is a like as sentence and def compute repetation word ratio\n
    repetation_ratio("yok bir şey yok bir şey yok bir şey") 
    '''
    word_list = word_tokenize(text)
    word_num = len(word_list)
    word_unique_num = len(set(word_list))
    ratio = int((word_unique_num/word_num)*100)
    
    return ratio

In [10]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987921,karneleme,5
987922,karnaya,5
987923,dörtlümüzün,5
987924,karnavalınız,5


In [11]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [12]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
995,hale,70625
996,kavga,70611
997,arabayı,70310
998,oldun,70121


In [13]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [14]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [15]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

1000

#### Youtube Sentence Analysis

In [262]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.000,00:00:04.000,مرحبا أصدقائي,Qx98dvR61cY
1,00:00:06.000,00:00:09.017,مضى اكثر من اسبوعين,Qx98dvR61cY
2,00:00:09.017,00:00:11.517,عندما هطلت الأمطار,Qx98dvR61cY
3,00:00:11.550,00:00:13.667,وحدثت فيضانات,Qx98dvR61cY
4,00:00:13.684,00:00:20.667,المعروف والشائع أن لون السيول اصفر طيني,Qx98dvR61cY
...,...,...,...,...
918116,00:00:34.396,00:00:36.396,قولتله خلاص سيبها,Hp54ZaJS1nI
918117,00:00:36.396,00:00:40.240,و حضنتها زى مانتم شايفين كده,Hp54ZaJS1nI
918118,00:00:45.588,00:00:48.627,حسيت بمشاعر جامدة جدا و هى فى حضنى,Hp54ZaJS1nI
918119,00:00:48.835,00:00:50.835,كأنك شايل طفل بالظبط,Hp54ZaJS1nI


In [263]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [264]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,0.000,4.000,مرحبا أصدقائي,Qx98dvR61cY
1,1,6.000,9.017,مضى اكثر من اسبوعين,Qx98dvR61cY
2,2,9.017,11.517,عندما هطلت الأمطار,Qx98dvR61cY
3,3,11.550,13.667,وحدثت فيضانات,Qx98dvR61cY
4,4,13.684,20.667,المعروف والشائع أن لون السيول اصفر طيني,Qx98dvR61cY
...,...,...,...,...,...
918116,918116,34.396,36.396,قولتله خلاص سيبها,Hp54ZaJS1nI
918117,918117,36.396,40.240,و حضنتها زى مانتم شايفين كده,Hp54ZaJS1nI
918118,918118,45.588,48.627,حسيت بمشاعر جامدة جدا و هى فى حضنى,Hp54ZaJS1nI
918119,918119,48.835,50.835,كأنك شايل طفل بالظبط,Hp54ZaJS1nI


In [265]:
#df_youtube_sentence[df_youtube_sentence["index"] == 1287503]

In [266]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/(len(sent_word)+0.001))*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [267]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id","word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.000,4.000,مرحبا أصدقائي,Qx98dvR61cY,0.0,"أصدقائي, مرحبا",
1,1,6.000,9.017,مضى اكثر من اسبوعين,Qx98dvR61cY,25.0,"مضى, اسبوعين, اكثر",من
2,2,9.017,11.517,عندما هطلت الأمطار,Qx98dvR61cY,0.0,"هطلت, عندما, الأمطار",
3,3,11.550,13.667,وحدثت فيضانات,Qx98dvR61cY,0.0,"وحدثت, فيضانات",
4,4,13.684,20.667,المعروف والشائع أن لون السيول اصفر طيني,Qx98dvR61cY,14.3,"السيول, المعروف, والشائع, اصفر, لون, طيني",أن
...,...,...,...,...,...,...,...,...
918116,918116,34.396,36.396,قولتله خلاص سيبها,Hp54ZaJS1nI,0.0,"سيبها, قولتله, خلاص",
918117,918117,36.396,40.240,و حضنتها زى مانتم شايفين كده,Hp54ZaJS1nI,16.7,"زى, حضنتها, شايفين, مانتم, كده",و
918118,918118,45.588,48.627,حسيت بمشاعر جامدة جدا و هى فى حضنى,Hp54ZaJS1nI,12.5,"بمشاعر, هى, جدا, حضنى, حسيت, جامدة, فى",و
918119,918119,48.835,50.835,كأنك شايل طفل بالظبط,Hp54ZaJS1nI,0.0,"كأنك, شايل, بالظبط, طفل",


In [268]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.000,4.000,مرحبا أصدقائي,Qx98dvR61cY,0.0,"أصدقائي, مرحبا",NaN
1,1,6.000,9.017,مضى اكثر من اسبوعين,Qx98dvR61cY,25.0,"مضى, اسبوعين, اكثر",من
2,2,9.017,11.517,عندما هطلت الأمطار,Qx98dvR61cY,0.0,"هطلت, عندما, الأمطار",NaN
3,3,11.550,13.667,وحدثت فيضانات,Qx98dvR61cY,0.0,"وحدثت, فيضانات",NaN
4,4,13.684,20.667,المعروف والشائع أن لون السيول اصفر طيني,Qx98dvR61cY,14.3,"السيول, المعروف, والشائع, اصفر, لون, طيني",أن
...,...,...,...,...,...,...,...,...
918116,918116,34.396,36.396,قولتله خلاص سيبها,Hp54ZaJS1nI,0.0,"سيبها, قولتله, خلاص",NaN
918117,918117,36.396,40.240,و حضنتها زى مانتم شايفين كده,Hp54ZaJS1nI,16.7,"زى, حضنتها, شايفين, مانتم, كده",و
918118,918118,45.588,48.627,حسيت بمشاعر جامدة جدا و هى فى حضنى,Hp54ZaJS1nI,12.5,"بمشاعر, هى, جدا, حضنى, حسيت, جامدة, فى",و
918119,918119,48.835,50.835,كأنك شايل طفل بالظبط,Hp54ZaJS1nI,0.0,"كأنك, شايل, بالظبط, طفل",NaN


In [269]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result["word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,1,6.000,9.017,مضى اكثر من اسبوعين,Qx98dvR61cY,25.0,"مضى, اسبوعين, اكثر",من
1,7,40.434,44.400,وهذه المياه من اين تأتي لا أحد يعرف,Qx98dvR61cY,25.0,"المياه, يعرف, وهذه, تأتي, اين, أحد","لا, من"
2,8,50.400,53.000,هذا مجرى المياه,Qx98dvR61cY,33.3,"المياه, مجرى",هذا
3,10,113.100,119.284,ملاحظه أصدقائي هذه قناتي الجديده اشتركو فيها م...,Qx98dvR61cY,20.0,"ملاحظه, الجديده, تخسرو, شي, فيها, قناتي, أصدقا...","ما, هذه"
4,11,0.725,2.285,أهلا وسهلا فيكم وين ما كنتو,zRkO1P6_fZM,16.7,"كنتو, فيكم, وين, أهلا, وسهلا",ما
...,...,...,...,...,...,...,...,...
362910,918074,544.363,546.828,جاهل في سن الصبى والطيش,_bodYtWV23A,20.0,"الصبى, سن, والطيش, جاهل",في
362911,918076,550.507,552.864,جاهل في سن الصبى والطيش,_bodYtWV23A,20.0,"الصبى, سن, والطيش, جاهل",في
362912,918105,0.979,2.979,طلعت تجرى و جت عليا,Hp54ZaJS1nI,20.0,"طلعت, تجرى, جت, عليا",و
362913,918113,27.618,29.618,و انا حبيت حبوبة اوى,Hp54ZaJS1nI,20.0,"اوى, انا, حبيت, حبوبة",و


In [ ]:
# will test
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list4 = manager.list()
index_list = manager.list()
all_index_list = manager.list()
var_index_list = manager.list()
text_all_list = manager.list()  
df = df_adjust_sentence_ratio
df.reset_index(drop=True, inplace=True)
index_num = list(range(len(df)))
sent_len = word_group_adjust_select
sent_len_num = sentence_word_count_number 

def df_exract_word_group(index_num):    
    source_text = df.loc[index_num,"sentence"]
    opt_var = df.loc[index_num,"video_id"]
    words = re.findall(r"\w+", source_text, re.UNICODE)
    # check words in word_list
    index_list = []
    for j in range(len(words)):
        if words[j] in word_list:
            index_list.append(j)
        else:
            pass
    # add index plus 1 and compare with current and next index (Is it continuous or not?)
    all_index_list = []
    var_index_list = []
    for k in range(len(index_list)):
        try:
            var1 = index_list[k] + 1  
            var2 = index_list[k+1]
        except:
            var1 = index_list[k] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[k]
            var_index_list.append(var3)
        else:
            var3 = index_list[k]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []
    # index convert to word
    text_all_list = []
    for m in all_index_list:
        text_list = [] 
        for n in m:
            word = words[n]
            text_list.append(word)
            if sent_len:
                if len(text_list) >= sent_len_num:
                    text = " ".join(text_list)
                    text_all_list.append(text)
                else:
                    pass
            else:
               text = " ".join(text_list)
               text_all_list.append(text) 
    # result list    
    for search_string in text_all_list:
        #df_var["index"] = i
        #df_var["search_string"] = search_string
        #df_var["sentence"] = source_text
        #df_var["video_id"] = opt_var
        result_list4.append([index_num,search_string,source_text,opt_var])

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(df_exract_word_group, index_num)

In [ ]:
df_adjust_sentence_ratio = pd.DataFrame(list(result_list4), columns=["index","search_string","sentence","video_id"])
df_adjust_sentence_ratio.sort_values(by="index", ascending=True, inplace=True)
df_adjust_sentence_ratio.reset_index(drop=True, inplace=True)
df_adjust_sentence_ratio.drop(["index"], axis=1, inplace=True)
df_adjust_sentence_ratio

In [270]:
#df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
#df_adjust_sentence_ratio = df_adjust_sentence_ratio[~(df_adjust_sentence_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
##df_adjust_sentence_ratio.reset_index(drop=True, inplace=True)
#df_adjust_sentence_ratio

<ipython-input-270-ca15520b3270>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))


,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word,search_string
19,45,21.190,22.618,علا و لكن ماذا لو أردت ذلك أنا,8mAA3d5GLSU,50.0,"لكن, أردت, لو, علا","أنا, ذلك, ماذا, و",ذلك أنا
21,47,23.611,25.799,أنا ياسين و أنت علا,8mAA3d5GLSU,60.0,"ياسين, علا","أنت, أنا, و",و أنت
22,48,32.517,36.080,بالمناسبة قناتنا جديدة و هذا هو أول فيديو لنا,8mAA3d5GLSU,33.3,"أول, قناتنا, بالمناسبة, لنا, فيديو, جديدة","هذا, هو, و",و هذا هو
35,68,129.074,132.604,و نخبركم عن تجربتنا عن هذا المطعم,8mAA3d5GLSU,42.9,"نخبركم, تجربتنا, المطعم","عن, هذا, و",عن هذا
44,88,213.681,216.989,أظن أن هذه هي طاسة الحمام,8mAA3d5GLSU,33.3,"أظن, طاسة, الحمام, هي","هذه, أن",أن هذه
...,...,...,...,...,...,...,...,...,...
362861,917878,12.596,16.204,و كل الناس اللى جايين جنينة الحيوانات جايين يش...,R0JvudmWrG0,20.0,"جايين, فقط, جنينة, الحيوانات, الناس, اللى, يشو...","كل, و",و كل
362863,917881,26.263,28.263,و هنا زودتها حبيتين,R0JvudmWrG0,50.0,"زودتها, حبيتين","هنا, و",و هنا
362876,917918,63.268,67.228,تعملهم بعد ما ان انت خلاص حققت الشروط خلاص كده...,9wtYwN0VaKA,15.4,"بعد, الشروط, خلاص, هتبدأ, كده, حققت, انت, تعملهم","ما, ان",ما ان
362879,917925,85.708,91.268,اللي هو تحقيق الربح دي قناة ليست مفعلة بتلاقي ...,9wtYwN0VaKA,25.0,"اللي, دي, مفعلة, بتلاقي, تحقيق, قناة, ليست, ال...","هنا, هو, في",هنا في


In [271]:
df_word_group_time_loc_in_sent_result = word_group_time_loc(df_adjust_sentence_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,ذلك أنا,22.237200,22.618000,علا و لكن ماذا لو أردت ذلك أنا,8mAA3d5GLSU
1,و أنت,24.647421,25.453526,أنا ياسين و أنت علا,8mAA3d5GLSU
2,و هذا هو,34.258911,35.050689,بالمناسبة قناتنا جديدة و هذا هو أول فيديو لنا,8mAA3d5GLSU
3,عن هذا,131.106424,131.962182,و نخبركم عن تجربتنا عن هذا المطعم,8mAA3d5GLSU
4,أن هذه,214.077960,215.136520,أظن أن هذه هي طاسة الحمام,8mAA3d5GLSU
...,...,...,...,...,...
64274,و كل,12.596000,12.924000,و كل الناس اللى جايين جنينة الحيوانات جايين يش...,R0JvudmWrG0
64275,و هنا,26.263000,26.894579,و هنا زودتها حبيتين,R0JvudmWrG0
64276,ما ان,63.939186,64.409017,تعملهم بعد ما ان انت خلاص حققت الشروط خلاص كده...,9wtYwN0VaKA
64277,هنا في,90.585193,91.268000,اللي هو تحقيق الربح دي قناة ليست مفعلة بتلاقي ...,9wtYwN0VaKA


In [272]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,ذلك أنا,21.937200,22.918000,علا و لكن ماذا لو أردت ذلك أنا,8mAA3d5GLSU
1,و أنت,24.347421,25.753526,أنا ياسين و أنت علا,8mAA3d5GLSU
2,و هذا هو,33.958911,35.350689,بالمناسبة قناتنا جديدة و هذا هو أول فيديو لنا,8mAA3d5GLSU
3,عن هذا,130.806424,132.262182,و نخبركم عن تجربتنا عن هذا المطعم,8mAA3d5GLSU
4,أن هذه,213.777960,215.436520,أظن أن هذه هي طاسة الحمام,8mAA3d5GLSU
...,...,...,...,...,...
64274,و كل,12.296000,13.224000,و كل الناس اللى جايين جنينة الحيوانات جايين يش...,R0JvudmWrG0
64275,و هنا,25.963000,27.194579,و هنا زودتها حبيتين,R0JvudmWrG0
64276,ما ان,63.639186,64.709017,تعملهم بعد ما ان انت خلاص حققت الشروط خلاص كده...,9wtYwN0VaKA
64277,هنا في,90.285193,91.568000,اللي هو تحقيق الربح دي قناة ليست مفعلة بتلاقي ...,9wtYwN0VaKA


In [273]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,ذلك أنا,22,23,علا و لكن ماذا لو أردت ذلك أنا,8mAA3d5GLSU
1,و أنت,24,26,أنا ياسين و أنت علا,8mAA3d5GLSU
2,و هذا هو,34,35,بالمناسبة قناتنا جديدة و هذا هو أول فيديو لنا,8mAA3d5GLSU
3,عن هذا,131,132,و نخبركم عن تجربتنا عن هذا المطعم,8mAA3d5GLSU
4,أن هذه,214,215,أظن أن هذه هي طاسة الحمام,8mAA3d5GLSU
...,...,...,...,...,...
64274,و كل,12,13,و كل الناس اللى جايين جنينة الحيوانات جايين يش...,R0JvudmWrG0
64275,و هنا,26,27,و هنا زودتها حبيتين,R0JvudmWrG0
64276,ما ان,64,65,تعملهم بعد ما ان انت خلاص حققت الشروط خلاص كده...,9wtYwN0VaKA
64277,هنا في,90,92,اللي هو تحقيق الربح دي قناة ليست مفعلة بتلاقي ...,9wtYwN0VaKA


In [274]:
df_word_group_time_loc_in_sent_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_sent_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,نعم ماذا عن هذا هنا نعم,381,391,نعم ماذا عن هذا هنا نعم,2l3UajCV1mI
1,ماذا ماذا ماذا أنت ما,882,885,ماذا ماذا ماذا أنت ما هي لا لا تفعل أنا آسف اا...,HnSK6tiiQZg
2,من هذا هل هذه أنت يا,292,295,من هذا هل هذه أنت يا حفيدتي العزيزة,FxiTk2FcyHk
3,من هذا هل هذه أنت يا,800,803,من هذا هل هذه أنت يا حفيدتي العزيزة,3F6yc6Ly8Pc
4,من أنت من أنت من أنت,594,597,التبرير وعلى التفاعل فقط اذن من أنت من أنت من ...,J5I9JQLXplQ
...,...,...,...,...,...
64274,و و,305,307,اللي هي و و,6RFyHzrP1pA
64275,و و,160,161,فالتفت إليهم و و أخرج من كنانته,kwAxcrc9Be8
64276,و و,327,328,تم بث هذه المناظرات على قنوات و و لتحقق نسبة م...,IiE3UmLRkuE
64277,و و,18,20,و و وغيرها,B35kMVoUcbc


In [275]:
df_word_group_time_loc_in_sent_result["unique_word_ratio"] = df_word_group_time_loc_in_sent_result["search_string"].apply(lambda text: repetation_ratio(text))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id,unique_word_ratio
0,نعم ماذا عن هذا هنا نعم,381,391,نعم ماذا عن هذا هنا نعم,2l3UajCV1mI,83
1,ماذا ماذا ماذا أنت ما,882,885,ماذا ماذا ماذا أنت ما هي لا لا تفعل أنا آسف اا...,HnSK6tiiQZg,60
2,من هذا هل هذه أنت يا,292,295,من هذا هل هذه أنت يا حفيدتي العزيزة,FxiTk2FcyHk,100
3,من هذا هل هذه أنت يا,800,803,من هذا هل هذه أنت يا حفيدتي العزيزة,3F6yc6Ly8Pc,100
4,من أنت من أنت من أنت,594,597,التبرير وعلى التفاعل فقط اذن من أنت من أنت من ...,J5I9JQLXplQ,33
...,...,...,...,...,...,...
64274,و و,305,307,اللي هي و و,6RFyHzrP1pA,50
64275,و و,160,161,فالتفت إليهم و و أخرج من كنانته,kwAxcrc9Be8,50
64276,و و,327,328,تم بث هذه المناظرات على قنوات و و لتحقق نسبة م...,IiE3UmLRkuE,50
64277,و و,18,20,و و وغيرها,B35kMVoUcbc,50


In [276]:
df_word_group_time_loc_in_sent_result_repeat_ratio = df_word_group_time_loc_in_sent_result[df_word_group_time_loc_in_sent_result["unique_word_ratio"] > unique_word_ratio]
df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-276-d6186b820154>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
<ipython-input-276-d6186b820154>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)


,search_string,start_time,end_time,sentence,video_id
0,نعم ماذا عن هذا هنا نعم,381,391,نعم ماذا عن هذا هنا نعم,2l3UajCV1mI
1,من هذا هل هذه أنت يا,292,295,من هذا هل هذه أنت يا حفيدتي العزيزة,FxiTk2FcyHk
2,من هذا هل هذه أنت يا,800,803,من هذا هل هذه أنت يا حفيدتي العزيزة,3F6yc6Ly8Pc
3,من هذا هل هذه أنت يا,1587,1590,من هذا هل هذه أنت يا حفيدتي العزيزة,DTPTK0j5dZY
4,هل كان هذا كل ما في,366,368,هل كان هذا كل ما في الأمر,RLydD3uP6NI
...,...,...,...,...,...
55421,و لا,69,70,لا اجيكم و لا تجوني,_hvmySpIUcM
55422,و هو,473,474,و هو عادة السكر الصائم أقل من,rw9Zd97PitI
55423,و يا,1022,1023,و يا الله السلامة و الله لا يغير علينا و الأمث...,CcdyhmjNiiU
55424,و لا,93,95,لسه ما وجدنا و لا أحد,y9fjnZNRS6c


In [277]:
df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-277-e137816795b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)


,search_string,start_time,end_time,sentence,video_id,include_string_ratio
0,نعم ماذا عن هذا هنا نعم,381,391,نعم ماذا عن هذا هنا نعم,2l3UajCV1mI,100
1,من هذا هل هذه أنت يا,292,295,من هذا هل هذه أنت يا حفيدتي العزيزة,FxiTk2FcyHk,57
2,من هذا هل هذه أنت يا,800,803,من هذا هل هذه أنت يا حفيدتي العزيزة,3F6yc6Ly8Pc,57
3,من هذا هل هذه أنت يا,1587,1590,من هذا هل هذه أنت يا حفيدتي العزيزة,DTPTK0j5dZY,57
4,هل كان هذا كل ما في,366,368,هل كان هذا كل ما في الأمر,RLydD3uP6NI,76
...,...,...,...,...,...,...
55421,و لا,69,70,لا اجيكم و لا تجوني,_hvmySpIUcM,21
55422,و هو,473,474,و هو عادة السكر الصائم أقل من,rw9Zd97PitI,13
55423,و يا,1022,1023,و يا الله السلامة و الله لا يغير علينا و الأمث...,CcdyhmjNiiU,6
55424,و لا,93,95,لسه ما وجدنا و لا أحد,y9fjnZNRS6c,19


In [278]:
df_word_group_time_loc_include_string_ratio = df_word_group_time_loc_in_sent_result_repeat_ratio[df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] > include_wordgroup_ratio]
df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)
df_word_group_time_loc_include_string_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_include_string_ratio

<ipython-input-278-ea709f90da2b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)


,search_string,start_time,end_time,sentence,video_id
0,نعم ماذا عن هذا هنا نعم,381,391,نعم ماذا عن هذا هنا نعم,2l3UajCV1mI
1,هل كان هذا كل ما في,366,368,هل كان هذا كل ما في الأمر,RLydD3uP6NI
2,عن ماذا كان كل ذلك,262,265,عن ماذا كان كل ذلك,rVkXdGORB8U
3,في في هل هذه أنت,4218,4220,مرحبا في في هل هذه أنت,7ONEprXyxcs
4,أنا هنا يا في في,4868,4870,أنا هنا يا في في,7ONEprXyxcs
...,...,...,...,...,...
1065,و لا,616,618,و لا,Mms3BuNkHBs
1066,و هو,53,58,و هو,KBaejvlMXfY
1067,و هو,172,175,و هو,VR6GurhR33A
1068,و في,53,56,و في,glrQ18CS24A


In [279]:
df_youtube_word_usage = word_usage_result(word_list, df_word_group_time_loc_include_string_ratio, "search_string", "video_id", word_usage_min, word_usage_max)
df_youtube_word_usage.drop_duplicates(inplace=True)
df_youtube_word_usage

,search_string,video_id
0,ما في,zscMzTqil5s
1,ليس هذا,zXRS7rQha0A
2,هناك من لا,zUdW5hQzRDw
3,لا يا,zUdW5hQzRDw
4,و هذا هو,ylMv8Xd7fGk
...,...,...
514,من أنا,2QUfHeWMjSQ
515,لا ليس هنا,1SsI1Hk275c
516,لا ليس هناك,1SsI1Hk275c
517,ليس أنت,02qFT9oP8jU


In [280]:
df_youtube_word_usage_merge = pd.merge(df_word_group_time_loc_include_string_ratio,df_youtube_word_usage,how="inner",on=["search_string","video_id"])
df_youtube_word_usage_merge.drop_duplicates(subset=["search_string","video_id"],inplace=True)
df_youtube_word_usage_merge.reset_index(drop=True, inplace=True)
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id
0,نعم ماذا عن هذا هنا نعم,381,391,نعم ماذا عن هذا هنا نعم,2l3UajCV1mI
1,هل كان هذا كل ما في,366,368,هل كان هذا كل ما في الأمر,RLydD3uP6NI
2,عن ماذا كان كل ذلك,262,265,عن ماذا كان كل ذلك,rVkXdGORB8U
3,في في هل هذه أنت,4218,4220,مرحبا في في هل هذه أنت,7ONEprXyxcs
4,أنا هنا يا في في,4868,4870,أنا هنا يا في في,7ONEprXyxcs
...,...,...,...,...,...
432,و لا,612,614,و لا,Mms3BuNkHBs
433,و هو,53,58,و هو,KBaejvlMXfY
434,و هو,172,175,و هو,VR6GurhR33A
435,و في,53,56,و في,glrQ18CS24A


In [281]:
df_youtube_word_usage_merge["video_url"] = "https://www.youtube.com/watch?v="+df_youtube_word_usage_merge['video_id'].map(str)+"&t="+df_youtube_word_usage_merge['start_time'].map(str)+"s"
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id,video_url
0,نعم ماذا عن هذا هنا نعم,381,391,نعم ماذا عن هذا هنا نعم,2l3UajCV1mI,https://www.youtube.com/watch?v=2l3UajCV1mI&t=...
1,هل كان هذا كل ما في,366,368,هل كان هذا كل ما في الأمر,RLydD3uP6NI,https://www.youtube.com/watch?v=RLydD3uP6NI&t=...
2,عن ماذا كان كل ذلك,262,265,عن ماذا كان كل ذلك,rVkXdGORB8U,https://www.youtube.com/watch?v=rVkXdGORB8U&t=...
3,في في هل هذه أنت,4218,4220,مرحبا في في هل هذه أنت,7ONEprXyxcs,https://www.youtube.com/watch?v=7ONEprXyxcs&t=...
4,أنا هنا يا في في,4868,4870,أنا هنا يا في في,7ONEprXyxcs,https://www.youtube.com/watch?v=7ONEprXyxcs&t=...
...,...,...,...,...,...,...
432,و لا,612,614,و لا,Mms3BuNkHBs,https://www.youtube.com/watch?v=Mms3BuNkHBs&t=...
433,و هو,53,58,و هو,KBaejvlMXfY,https://www.youtube.com/watch?v=KBaejvlMXfY&t=53s
434,و هو,172,175,و هو,VR6GurhR33A,https://www.youtube.com/watch?v=VR6GurhR33A&t=...
435,و في,53,56,و في,glrQ18CS24A,https://www.youtube.com/watch?v=glrQ18CS24A&t=53s


In [282]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(2000)
df_youtube_word_usage_merge.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.csv", index=False)

In [283]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(700)
df_youtube_word_usage_merge.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

In [284]:
#df_word_group_time_loc_in_sent_result = df_word_group_time_loc_in_sent_result.head(1000000)
#df_word_group_time_loc_in_sent_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [285]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.*")
output_file

['Arabic_Word_Group_Max_In_Youtube_Sentence_28_Word6.csv',
 'Arabic_Word_Group_Max_In_Youtube_Sentence_28_Word6.xlsx']

In [286]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [287]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass